### Test dynamic env class

In [ ]:
import time, sys
sys.path.append('..')
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
from pb_diff_envs.environment.static.maze2d_rand_wgrp import Maze2DRandRecGroupList
import pb_diff_envs.environment.dynamic.dyn_rm2d_env as dre; reload(dre)
from pb_diff_envs.environment.dynamic.dyn_rm2d_env import DynamicRandMaze2DEnv
no_while = True

# --- 2 ---
wall_locations = np.array(
    [np.array([1.0, 1.0]), np.array([4.2, 1.0]),])
maze_size = np.array([5, 5])
wall_hExts = np.array([[0.5, 0.5],[0.5, 0.5],])
len_full_wtraj = 12
num_sample_pts = 2
gap_betw_wall = 0.15
wp_seed = 100
speed = 0.12 # 0.2 too long?

robot_config = dict(maze_size=maze_size, min_to_wall_dist=0.01, collision_eps=0.05)
env = DynamicRandMaze2DEnv(wall_locations, wall_hExts, 
                 len_full_wtraj, 
                 num_sample_pts,
                 gap_betw_wall,
                robot_config, wp_seed, speed, renderer_config={})

## simple vis of env

In [ ]:
# env.robot.print_joint_states()
# print(env.robot.item_id)
# print(env.robot_id) # list
print(env.robot.limits_low)
print(env.robot.limits_high)
print(env._get_joint_pose())
print(f'{type(env._get_joint_pose())}')


In [ ]:
# env.resample_robot_start_pos()
print(env.recWall_grp.get_walls_pos())
print(env.recWall_grp.get_wtrajs_end_pos())
print(env.recWall_grp.get_walls_pos_at_t(0))
print(env.recWall_grp.is_walls_in_start_pos())
pts = []
for i in range(20000):
    pts.append(env.get_robot_free_pose())
pts = np.array(pts)
# img = env.render()
# img = env.render_1_traj(pts, env.recWall_grp.get_wtrajs(), img_type='png')
from utils.maze2d_utils import plt_img
# plt_img(img)
# env.render_1_traj(pts)
# env.recWall_grp.set_walls_at_time(0)
plt.scatter(pts[:, 0], pts[:, 1])
plt.gca().set_aspect('equal')
plt.xlim(-1, 6); plt.ylim(-1, 6)
# plt_img(img)

## Sample a problem/episode

In [ ]:
from utils.utils import DotDict
from tqdm import tqdm
from utils.utils import seed_everything
seed_everything(42) # 42
# env.load(GUI=True) # False
prev_pose = env.get_robot_free_pose()
prev_pose = np.array([2.93287797, 3.33783522])
env.robot.set_config(prev_pose)
solutions = []
wtrajs_list = []
# n_traj = 25
n_traj = 1
for i in tqdm(range(n_traj)):
    solution, d = env.sample_1_episode(prev_pose)

    wtrajs = d['wtrajs'] # n_w, h, 2
    wtrajs_list.append(wtrajs)

    solution = np.array(solution)
    prev_pose = solution[-1]
    solutions.append(solution)

## can concatenate all solutions
# solutions = np.concatenate(solutions, axis=0)
# wtrajs_list = np.concatenate(wtrajs_list, axis=1)
# result = DotDict(solution=solutions)
# print(solutions.shape) 
# print(len(result.solution))

In [ ]:
from glob import glob
savepath = './test_sipp_dyn_maze2d.gif'
sol_i = solutions[0]
wtrajs_i = wtrajs_list[0]
# gifs = env.render_1_traj(solutions, savepath=savepath, wtrajs=wtrajs_list) ## visualize all if concat
gifs = env.render_1_traj(sol_i, savepath=savepath, wtrajs=wtrajs_i)
savepath = glob(f"{savepath.replace('.gif','')}**")[0] ## retrieve the gif path
###
print(len(gifs), gifs[0].shape)
from IPython.display import HTML
import base64
b64 = base64.b64encode(open(savepath, 'rb').read()).decode('ascii')
display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))

In [ ]:
print(solution.shape) # (, 2)
